In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import re

from sklearn.linear_model import LinearRegression
import seaborn
import pyperclip


In [47]:
pd.set_option('display.max_columns', 50)

### ここだけ手動で設定。変更がないか毎度確認すること

In [56]:
subject_name_list = ["kumakura","kim","souma","fujii","tubota","toki"]
phase_name_list = ["rest", "practice", "boredom", "flow", "ultra", "overload"]
target_columns = ["bpm", "ibi", "lf", "hf","lf/hf","sdsd","rmssd","pnn20","pnn50","concentration_FC3","concentration_FC4","concentration_FCz","concentration_mean","FC3","FC4","FCz","mean_3ch","lf_diff", "hf_diff","lfhf_diff","hr_mad","sd1","sd2","s","sd1/sd2","breathingrate","lf_nu","hf_nu"]
target_phases = ["boredom","flow","ultra","overload"]

folder_name = "999_1_sandbox"
pyperclip.copy(f"{folder_name}")


### 分析対象のファイルのパスを正規表現で取得

In [57]:
pathes = glob.glob("/Users/miyakooti/repositories/arai_MATLAB_program/csv/?_*/HRV_and_PLI.csv")
pathes.sort()
pathes

['/Users/miyakooti/repositories/arai_MATLAB_program/csv/0_kumakura/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/1_kim/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/2_souma/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/3_fujii/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/4_tubota/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/5_toki/HRV_and_PLI.csv']

# StatsModelsを利用した分析

- [公式ドキュメント](https://www.statsmodels.org/stable/index.html#citation)
- [github](https://github.com/statsmodels/statsmodels)

Seabold, Skipper, and Josef Perktold. “statsmodels: Econometric and statistical modeling with python.” Proceedings of the 9th Python in Science Conference. 2010.

In [58]:
import statsmodels.api as sm

### フェーズごとの分析

In [59]:
for i,target_phase in enumerate(target_phases):
    
    for j,path in enumerate(pathes):

        df = pd.read_csv(path,index_col=0).fillna(0)
        # column
        # row
        df = df.loc[[target_phase]] # seriesとして取り出したいときはこっち
        if j == 0:
            flow_dataset = df
        else:
            flow_dataset = pd.concat([flow_dataset, df], axis=0)
    
    export_data = {    
        "target": target_columns,
        "linear-p": [],
        "linear-rsquared": [],
        "linear-rsquared_adj": [],
        "linear-coef": [],
        "nonlinear-p": [],
        "nonlinear-rsquared": [],
        "nonlinear-rsquared_adj": [],
        "nonlinear-coef": [],
    }

    for target_column in target_columns:
        
        x = flow_dataset[target_column]
        y = flow_dataset[["questionnaire_average"]]
        
        for k in ["linear", "nonlinear"]:
            if k=="linear":
                # 線形単回帰
                X = sm.add_constant(x)
                model = sm.OLS(y, X)
                results = model.fit()

                export_data["linear-p"].append(results.pvalues[target_column])
                export_data["linear-rsquared"].append(results.rsquared)
                export_data["linear-rsquared_adj"].append(results.rsquared_adj)
                export_data["linear-coef"].append(results.params[target_column])
            if k=="nonlinear":
                # ２次単回帰
                x = x**2
                
                X = sm.add_constant(x)
                model = sm.OLS(y, X)
                results = model.fit()

                export_data["nonlinear-p"].append(results.pvalues[target_column])
                export_data["nonlinear-rsquared"].append(results.rsquared)
                export_data["nonlinear-rsquared_adj"].append(results.rsquared_adj)
                export_data["nonlinear-coef"].append(results.params[target_column])
    
    print(f"\nsaving {target_phase} phase regression results...\n")
    df = pd.DataFrame(export_data)   
    print(df)
    save_path = f"/Users/miyakooti/repositories/arai_MATLAB_program/csv/{folder_name}/{i}_{target_phase}_regression.csv"
    df.to_csv(save_path)  


saving boredom phase regression results...

                target  linear-p  linear-rsquared  linear-rsquared_adj  \
0                  bpm  0.553311     9.454578e-02            -0.131818   
1                  ibi  0.476457     1.334252e-01            -0.083218   
2                   lf  0.309313     2.528466e-01             0.066058   
3                   hf  0.473861     1.348897e-01            -0.081388   
4                lf/hf  0.999453     1.328616e-07            -0.250000   
5                 sdsd  0.135859     4.646965e-01             0.330871   
6                rmssd  0.103472     5.247443e-01             0.405930   
7                pnn20  0.048935     6.617648e-01             0.577206   
8                pnn50  0.152216     4.379964e-01             0.297496   
9    concentration_FC3  0.208504     3.593707e-01             0.199213   
10   concentration_FC4  0.218494     3.471302e-01             0.183913   
11   concentration_FCz  0.199906     3.702718e-01             0.212


saving overload phase regression results...

                target  linear-p  linear-rsquared  linear-rsquared_adj  \
0                  bpm  0.379504         0.195856            -0.005180   
1                  ibi  0.392464         0.186516            -0.016855   
2                   lf  0.584396         0.081092            -0.148635   
3                   hf  0.203989         0.365051             0.206313   
4                lf/hf  0.551365         0.095429            -0.130713   
5                 sdsd  0.047656         0.665898             0.582373   
6                rmssd  0.037643         0.700737             0.625921   
7                pnn20  0.058420         0.632919             0.541148   
8                pnn50  0.025853         0.749393             0.686742   
9    concentration_FC3  0.070402         0.600212             0.500264   
10   concentration_FC4  0.911672         0.003476            -0.245656   
11   concentration_FCz  0.462770         0.141264            -0.07

# 被験者ごとの分析

In [60]:
def findSubjectName(path):
    for subject_name in subject_name_list:
        if subject_name in path:
            return subject_name

In [61]:
for i,path in enumerate(pathes):
    if "kim" in path:
        continue
    df = pd.read_csv(path,index_col=0).fillna(0)
    
    # ここだけ設定してください。ごめんね
    flow_dataset = df.iloc[2:6]
    df = df[["bpm", "ibi", "lf", "hf","lf/hf","FC3","FC4","FCz","mean_3ch","concentration_FC3","concentration_FC4","concentration_FCz","questionnaire_average"]]

    
    export_data = {
        "target": target_columns,
        "linear-p": [],
        "linear-rsquared": [],
        "linear-rsquared_adj": [],
        "linear-coef": [],
        "nonlinear-p": [],
        "nonlinear-rsquared": [],
        "nonlinear-rsquared_adj": [],
        "nonlinear-coef": [],
    }
    
    for target_column in target_columns:

            x = flow_dataset[target_column]
            y = flow_dataset[["questionnaire_average"]]

            for k in ["linear", "nonlinear"]:
                if k=="linear":
                    # 線形単回帰
                    X = sm.add_constant(x)
                    model = sm.OLS(y, X)
                    results = model.fit()

                    export_data["linear-p"].append(results.pvalues[target_column])
                    export_data["linear-rsquared"].append(results.rsquared)
                    export_data["linear-rsquared_adj"].append(results.rsquared_adj)
                    export_data["linear-coef"].append(results.params[target_column])
                if k=="nonlinear":
                    # ２次単回帰
                    x = x**2

                    X = sm.add_constant(x)
                    model = sm.OLS(y, X)
                    results = model.fit()

                    export_data["nonlinear-p"].append(results.pvalues[target_column])
                    export_data["nonlinear-rsquared"].append(results.rsquared)
                    export_data["nonlinear-rsquared_adj"].append(results.rsquared_adj)
                    export_data["nonlinear-coef"].append(results.params[target_column])
                    
    print(f"\nsaving {findSubjectName(path)} regression results...\n")    
    df = pd.DataFrame(export_data)   
    print(df)

    save_path = f"/Users/miyakooti/repositories/arai_MATLAB_program/csv/{folder_name}/subject/{i}_{findSubjectName(path)}_regression.csv"
    df.to_csv(save_path)  


saving kumakura regression results...

                target  linear-p  linear-rsquared  linear-rsquared_adj  \
0                  bpm  0.167306         0.693380             0.540070   
1                  ibi  0.162137         0.702014             0.553021   
2                   lf  0.492039         0.258025            -0.112963   
3                   hf  0.000353         0.999293             0.998940   
4                lf/hf  0.107461         0.796625             0.694938   
5                 sdsd  0.911507         0.007831            -0.488254   
6                rmssd  0.141272         0.737414             0.606122   
7                pnn20  0.124262         0.766918             0.650377   
8                pnn50  0.371433         0.395097             0.092646   
9    concentration_FC3  0.867187         0.017639            -0.473541   
10   concentration_FC4  0.684532         0.099520            -0.350720   
11   concentration_FCz  0.307624         0.479385             0.219077  

OSError: Cannot save file into a non-existent directory: '/Users/miyakooti/repositories/arai_MATLAB_program/csv/999_1_sandbox/subject'

https://atmarkit.itmedia.co.jp/ait/articles/2109/14/news024.html

# すべてのフェーズ，すべての被験者のデータをまとめて分析

In [62]:
# 使用するデータセットは一つなので，ここでまとめて作っちゃいましょう

for i,target_phase in enumerate(target_phases):
    
    for j,path in enumerate(pathes):

        df = pd.read_csv(path,index_col=0).fillna(0)
        # column
        # row
        df = df.loc[[target_phase]] # seriesとして取り出したいときはこっち
        if j == 0 and i == 0:
            print("flow_dataset初期化")
            flow_dataset = df
        else:
            flow_dataset = pd.concat([flow_dataset, df], axis=0)
        

for i,target_phase in enumerate(target_phases):


    export_data = {    
        "target": target_columns,
        "linear-p": [],
        "linear-rsquared": [],
        "linear-rsquared_adj": [],
        "linear-coef": [],
        "nonlinear-p": [],
        "nonlinear-rsquared": [],
        "nonlinear-rsquared_adj": [],
        "nonlinear-coef": [],
    }

    for target_column in target_columns:
        
        x = flow_dataset[target_column]
        y = flow_dataset[["questionnaire_average"]]
        
        for k in ["linear", "nonlinear"]:
            if k=="linear":
                # 線形単回帰
                X = sm.add_constant(x)
                model = sm.OLS(y, X)
                results = model.fit()

                export_data["linear-p"].append(results.pvalues[target_column])
                export_data["linear-rsquared"].append(results.rsquared)
                export_data["linear-rsquared_adj"].append(results.rsquared_adj)
                export_data["linear-coef"].append(results.params[target_column])
            if k=="nonlinear":
                # ２次単回帰
                x = x**2
                
                X = sm.add_constant(x)
                model = sm.OLS(y, X)
                results = model.fit()

                export_data["nonlinear-p"].append(results.pvalues[target_column])
                export_data["nonlinear-rsquared"].append(results.rsquared)
                export_data["nonlinear-rsquared_adj"].append(results.rsquared_adj)
                export_data["nonlinear-coef"].append(results.params[target_column])
    
    print(f"\nsaving {target_phase} phase regression results...\n")
    df = pd.DataFrame(export_data)   
    print(df)
    save_path = f"/Users/miyakooti/repositories/arai_MATLAB_program/csv/{folder_name}/all_regression.csv"
    df.to_csv(save_path)  

flow_dataset初期化

saving boredom phase regression results...

                target  linear-p  linear-rsquared  linear-rsquared_adj  \
0                  bpm  0.711149         0.006356            -0.038809   
1                  ibi  0.742627         0.005001            -0.040226   
2                   lf  0.058286         0.153520             0.115043   
3                   hf  0.240268         0.062103             0.019471   
4                lf/hf  0.705595         0.006614            -0.038540   
5                 sdsd  0.895668         0.000799            -0.044619   
6                rmssd  0.833489         0.002053            -0.043308   
7                pnn20  0.711217         0.006353            -0.038813   
8                pnn50  0.834108         0.002038            -0.043324   
9    concentration_FC3  0.066971         0.144434             0.105544   
10   concentration_FC4  0.795556         0.003117            -0.042196   
11   concentration_FCz  0.240272         0.062102  

# overloadを抜いたやつで分析してみる

In [63]:
# 使用するデータセットは一つなので，ここでまとめて作っちゃいましょう

for i,target_phase in enumerate(target_phases):
    
    if target_phase == "overload":
        continue
    
    for j,path in enumerate(pathes):

        df = pd.read_csv(path,index_col=0).fillna(0)
        # column
        # row
        df = df.loc[[target_phase]] # seriesとして取り出したいときはこっち
        if j == 0 and i == 0:
            print("flow_dataset初期化")
            flow_dataset = df
        else:
            flow_dataset = pd.concat([flow_dataset, df], axis=0)
        

for i,target_phase in enumerate(target_phases):


    export_data = {    
        "target": target_columns,
        "linear-p": [],
        "linear-rsquared": [],
        "linear-rsquared_adj": [],
        "linear-coef": [],
        "nonlinear-p": [],
        "nonlinear-rsquared": [],
        "nonlinear-rsquared_adj": [],
        "nonlinear-coef": [],
    }

    for target_column in target_columns:
        
        x = flow_dataset[target_column]
        y = flow_dataset[["questionnaire_average"]]
        
        for k in ["linear", "nonlinear"]:
            if k=="linear":
                # 線形単回帰
                X = sm.add_constant(x)
                model = sm.OLS(y, X)
                results = model.fit()

                export_data["linear-p"].append(results.pvalues[target_column])
                export_data["linear-rsquared"].append(results.rsquared)
                export_data["linear-rsquared_adj"].append(results.rsquared_adj)
                export_data["linear-coef"].append(results.params[target_column])
            if k=="nonlinear":
                # ２次単回帰
                x = x**2
                
                X = sm.add_constant(x)
                model = sm.OLS(y, X)
                results = model.fit()

                export_data["nonlinear-p"].append(results.pvalues[target_column])
                export_data["nonlinear-rsquared"].append(results.rsquared)
                export_data["nonlinear-rsquared_adj"].append(results.rsquared_adj)
                export_data["nonlinear-coef"].append(results.params[target_column])
    
    print(f"\nsaving {target_phase} phase regression results...\n")
    df = pd.DataFrame(export_data)   
    print(df)
    save_path = f"/Users/miyakooti/repositories/arai_MATLAB_program/csv/{folder_name}/all_regressiton_without_overload.csv"
    df.to_csv(save_path)  

flow_dataset初期化

saving boredom phase regression results...

                target  linear-p  linear-rsquared  linear-rsquared_adj  \
0                  bpm  0.519030         0.026455            -0.034392   
1                  ibi  0.538445         0.024105            -0.036888   
2                   lf  0.033227         0.253336             0.206669   
3                   hf  0.115139         0.147849             0.094590   
4                lf/hf  0.933420         0.000450            -0.062022   
5                 sdsd  0.732546         0.007502            -0.054529   
6                rmssd  0.774333         0.005287            -0.056883   
7                pnn20  0.924979         0.000572            -0.061893   
8                pnn50  0.700197         0.009515            -0.052390   
9    concentration_FC3  0.046380         0.225607             0.177207   
10   concentration_FC4  0.408147         0.043154            -0.016648   
11   concentration_FCz  0.105148         0.155688  


saving ultra phase regression results...

                target  linear-p  linear-rsquared  linear-rsquared_adj  \
0                  bpm  0.519030         0.026455            -0.034392   
1                  ibi  0.538445         0.024105            -0.036888   
2                   lf  0.033227         0.253336             0.206669   
3                   hf  0.115139         0.147849             0.094590   
4                lf/hf  0.933420         0.000450            -0.062022   
5                 sdsd  0.732546         0.007502            -0.054529   
6                rmssd  0.774333         0.005287            -0.056883   
7                pnn20  0.924979         0.000572            -0.061893   
8                pnn50  0.700197         0.009515            -0.052390   
9    concentration_FC3  0.046380         0.225607             0.177207   
10   concentration_FC4  0.408147         0.043154            -0.016648   
11   concentration_FCz  0.105148         0.155688             0.10291

# 線形重回帰分析（マルチこに気をつけよう）

In [41]:
x = flow_dataset[["bpm", "ibi","mean_3ch"]]
y = flow_dataset[["questionnaire_average"]]

#全要素が1の列を説明変数の先頭に追加,切片をつけるために必ず必要
X = sm.add_constant(x)
 
#モデルの設定
model = sm.OLS(y, X)
 
#回帰分析の実行
results = model.fit()
 
#結果の詳細を表示
print(results.summary())

## いい結果のように思えるが、多重共線性により偽の有意性が出てしまっている

                              OLS Regression Results                             
Dep. Variable:     questionnaire_average   R-squared:                       0.092
Model:                               OLS   Adj. R-squared:                 -0.156
Method:                    Least Squares   F-statistic:                    0.3710
Date:                   Mon, 23 Jan 2023   Prob (F-statistic):              0.776
Time:                           12:00:11   Log-Likelihood:                -28.735
No. Observations:                     15   AIC:                             65.47
Df Residuals:                         11   BIC:                             68.30
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         60.5765 

/Users/miyakooti/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1769: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


### 参考
- https://takacity.blog.fc2.com/blog-entry-305.html
- https://self-development.info/%E3%80%90%E5%88%9D%E5%BF%83%E8%80%85%E8%84%B1%E5%87%BA%E3%80%91statsmodels%E3%81%AB%E3%82%88%E3%82%8B%E9%87%8D%E5%9B%9E%E5%B8%B0%E5%88%86%E6%9E%90%E7%B5%90%E6%9E%9C%E3%81%AE%E8%A6%8B%E6%96%B9/
- https://teratail.com/questions/256310

In [11]:
# 決定係数
# 0.9以上	非常によい
# 0.7以上0.9未満	よい
# 0.5以上0.7未満	あまりよくない
# 0.5未満	悪い

# Dep. Variable:     questionnaire_average   R-squared:                       0.915（決定係数。説明変数が目的変数をどれくらい説明できるか）
# Model:                               OLS   Adj. R-squared:                  0.893（自由度調整済み決定係数）
# Method:                    Least Squares   F-statistic:                     42.82（F値）
# Date:                   Fri, 13 Jan 2023   Prob (F-statistic):            0.00282（F値の現れる確率）
# Time:                           12:17:03   Log-Likelihood:                 2.8630
# No. Observations:           データの行数   AIC:                            -1.726
# Df Residuals:               残差の自由度   BIC:                            -2.142
# Df Model:                   要因の自由度                                        
# Covariance Type:  nonrobust（変数間の相関関係）


#       coef（回帰係数。傾き）    std err（標準誤差）  t      P>|t|      [0.025  0.975]
# ------------------------------------------------------------------------------
# const          7.3625      0.174     42.246      0.000       6.879       7.846
# lf         -6.729e-06   1.03e-06     -6.544      0.003   -9.58e-06   -3.87e-06
# ==============================================================================
# Omnibus:                          nan   Durbin-Watson:                   1.765
# Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.156
# Skew:                          -0.179   Prob(JB):                        0.925
# Kurtosis:                       2.296   Cond. No.                     3.93e+05
# ==============================================================================

In [12]:
dir(results)

['HC0_se',
 'HC1_se',
 'HC2_se',
 'HC3_se',
 '_HCCM',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abat_diagonal',
 '_cache',
 '_data_attr',
 '_data_in_cache',
 '_get_robustcov_results',
 '_is_nested',
 '_use_t',
 '_wexog_singular_values',
 'aic',
 'bic',
 'bse',
 'centered_tss',
 'compare_f_test',
 'compare_lm_test',
 'compare_lr_test',
 'condition_number',
 'conf_int',
 'conf_int_el',
 'cov_HC0',
 'cov_HC1',
 'cov_HC2',
 'cov_HC3',
 'cov_kwds',
 'cov_params',
 'cov_type',
 'df_model',
 'df_resid',
 'diagn',
 'eigenvals',
 'el_test',
 'ess',
 'f_pvalue',
 'f_test',
 'fittedvalues',
 'fvalue',
 'get_influence',
 'get_prediction',
 'get_robustcov_results',
 'info_c

In [13]:
print(results.__doc__)


    Results class for for an OLS model.

    Parameters
    ----------
    model : RegressionModel
        The regression model instance.
    params : ndarray
        The estimated parameters.
    normalized_cov_params : ndarray
        The normalized covariance parameters.
    scale : float
        The estimated scale of the residuals.
    cov_type : str
        The covariance estimator used in the results.
    cov_kwds : dict
        Additional keywords used in the covariance specification.
    use_t : bool
        Flag indicating to use the Student's t in inference.
    **kwargs
        Additional keyword arguments used to initialize the results.

    See Also
    --------
    RegressionResults
        Results store for WLS and GLW models.

    Notes
    -----
    Most of the methods and attributes are inherited from RegressionResults.
    The special methods that are only available for OLS are:

    - get_influence
    - outlier_test
    - el_test
    - conf_int_el
    


In [14]:
dir(results.params)

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__redu

- [pandas dictionary出力](https://note.nkmk.me/python-pandas-to-csv/)
- [pandas dictionary出力](https://atmarkit.itmedia.co.jp/ait/articles/2109/14/news024.html)